In [1]:
# importing requisite libraries
from stockstats import StockDataFrame as sdf
import requests
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
import alpaca_trade_api as tradeapi
from pathlib import Path
import lib_copy

In [2]:
load_dotenv()

Python-dotenv could not parse statement starting at line 7
Python-dotenv could not parse statement starting at line 8


True

In [3]:
dataframe = lib_copy.fetch_ohlcv("JPM")

In [4]:
def adding_boll_kelt_ewma_dataframe(dataframe):
    lib_copy.bollinger_band_generator(dataframe)
    lib_copy.keltner_channel(dataframe)
    lib_copy.ewma(dataframe)

    return dataframe

In [5]:
adding_boll_kelt_ewma_dataframe(dataframe).tail(1)

,open,high,low,close,volume,bollinger_band_middle,bollinger_band_std,bollinger_band_upper,bollinger_band_lower,close_-1_s,tr,tr_14_smma,atr,kcmid,kcup,kclo,EMA9,EMA21
2020-12-31 15:45:00-05:00,126.72,127.29,126.71,127.0,55144,126.1932,0.359947,126.913095,125.473305,126.73,0.58,0.257104,0.257104,126.1932,126.450304,125.936096,126.536989,126.216456


In [6]:
lib_copy.signals_generator(dataframe).head(1)

,open,high,low,close,volume,bollinger_band_middle,bollinger_band_std,bollinger_band_upper,bollinger_band_lower,close_-1_s,...,tr_14_smma,atr,kcmid,kcup,kclo,EMA9,EMA21,squeeze,crossover,target
2020-01-02 14:15:00-05:00,140.575,140.77,140.525,140.73,20048,140.19825,0.420517,141.039285,139.357215,NaN,...,NaN,NaN,NaN,NaN,NaN,140.73,140.73,0.0,0.0,0.0


In [7]:
def window_data(dataframe, window_size, feature_col_number, target_col_number):
    X = []
    y = []

    for i in range (len(dataframe)-window_size):
        features = dataframe.iloc[i: (i + window_size), feature_col_number]

        target = dataframe.iloc[(i + window_size), target_col_number]

        X.append(features)
        y.append(features)

    return np.array(X), np.array(y).reshape(-1,1) 

In [8]:
"""

columns = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
f_col = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
t_col = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
for b in columns:
    window_size = 18
    f_col = b
    t_col = b

    x,y = window_data(dataframe, window_size, f_col, t_col)
"""

'\n\ncolumns = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]\nf_col = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]\nt_col = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]\nfor b in columns:\n    window_size = 18\n    f_col = b\n    t_col = b\n\n    x,y = window_data(dataframe, window_size, f_col, t_col)\n'

In [9]:
def open_practice(dataframe, target_column = 1, feature_column = 1, window_size = 30):

    X, y = window_data(dataframe, window_size, feature_column, target_column)
    # Manually splitting the data
    split = int(0.7 * len(X))

    X_train = X[: split]
    X_test = X[split:]

    y_train = y[: split]
    y_test = y[split:]

    # Importing the MinMaxScaler from sklearn
    from sklearn.preprocessing import MinMaxScaler

    # Create a MinMaxScaler object
    scaler = MinMaxScaler()

    # Fit the MinMaxScaler object with the features data X
    scaler.fit(X)

    # Scale the features training and testing sets
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    # Fit the MinMaxScaler object with the target data Y
    scaler.fit(y)

    # Scale the target training and testing sets
    y_train = scaler.transform(y_train)
    y_test = scaler.transform(y_test)

    # Reshape the features data
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

    # Importing required Keras modules
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense, Dropout

    # Define the LSTM RNN model.
    model = Sequential()

    # Initial model setup
    number_units = 30
    dropout_fraction = 0.2

    # Layer 1
    model.add(LSTM(
        units=number_units,
        return_sequences=True,
        input_shape=(X_train.shape[1], 1))
        )
    model.add(Dropout(dropout_fraction))

    # Layer 2
    model.add(LSTM(units=number_units, return_sequences=True))
    model.add(Dropout(dropout_fraction))

    # Layer 3
    model.add(LSTM(units=number_units))
    model.add(Dropout(dropout_fraction))

    # Output layer
    model.add(Dense(1))

    # Compile the model
    model.compile(optimizer="adam", loss="mean_squared_error")

    model.summary() 

    # Train the model
    model.fit(X_train, y_train, epochs=10, shuffle=False, batch_size=90, verbose=1)

    # Evaluate the model
    model.evaluate(X_test, y_test, verbose=0)

    # Make predictions using the testing data X_test
    predicted = model.predict(X_test) 

    # Recover the original prices instead of the scaled version
    predicted_prices = scaler.inverse_transform(predicted)
    real_prices = scaler.inverse_transform(y_test.reshape(-1, 1)) 

    # Create a DataFrame of Real and Predicted values
    stocks = pd.DataFrame({
        "Actual": real_prices.ravel(),
        "Predicted": predicted_prices.ravel()
    }, index = dataframe.index[-len(real_prices): ]) 

    # Show the DataFrame's head
    stocks.head()

    #return model.evaluate(X_test, y_test, verbose=0), stocks.head()
    return stocks.head()

In [10]:
open_practice(dataframe)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 30)            3840      
_________________________________________________________________
dropout (Dropout)            (None, 30, 30)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 30)            7320      
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 30)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 30)                7320      
_________________________________________________________________
dropout_2 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 3

ValueError: Data cardinality is ambiguous:
  x sizes: 2021
  y sizes: 197307
Please provide data which shares the same first dimension.

In [11]:
split = int(0.7 * len(X))

X_train = X[: split]
X_test = X[split:]

y_train = y[: split]
y_test = y[split:] 

NameError: name 'X' is not defined

In [12]:
from sklearn.preprocessing import MinMaxScaler

# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Fit the MinMaxScaler object with the features data X
scaler.fit(X)

# Scale the features training and testing sets
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Fit the MinMaxScaler object with the target data Y
scaler.fit(y)

# Scale the target training and testing sets
y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)

NameError: name 'X' is not defined

In [13]:
# Reshape the features data
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Print some sample data after reshaping the datasets
print (f"X_train sample values:\n{X_train[:3]} \n")
print (f"X_test sample values:\n{X_test[:3]}")

NameError: name 'X_train' is not defined

In [14]:
 # Importing required Keras modules
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [15]:
# Define the LSTM RNN model.
model = Sequential()

# Initial model setup
number_units = 30
dropout_fraction = 0.2

# Layer 1
model.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train.shape[1], 1))
    )
model.add(Dropout(dropout_fraction))

# Layer 2
model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))

# Layer 3
model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))

# Output layer
model.add(Dense(1))

NameError: name 'X_train' is not defined

In [16]:
# Compile the model
model.compile(optimizer="adam", loss="mean_squared_error") 

In [17]:
 # Show the model summary
model.summary() 

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [18]:
model.fit(X_train, y_train, epochs=10, shuffle=False, batch_size=90, verbose=1) 

NameError: name 'X_train' is not defined

In [19]:
# Evaluate the model
model.evaluate(X_test, y_test, verbose=0)

NameError: name 'X_test' is not defined

In [20]:
predicted = model.predict(X_test) 

NameError: name 'X_test' is not defined

In [21]:
 # Recover the original prices instead of the scaled version
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

NameError: name 'predicted' is not defined

In [22]:
# Create a DataFrame of Real and Predicted values
big = pd.DataFrame({
    "Actual": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
}, index = dataframe.index[-len(real_prices): ]) 

# Show the DataFrame's head
big.head()

NameError: name 'real_prices' is not defined